# Estudo de Caso 2 - Processo de Inicialização e Otimização

Este Jupyter Notebook tem como objetivo a demonstração prática dos efeitos dos diferentes processos de inicialização e otimização para modelos com redes neurais. 

O processo de otimização em Deep Learning é o processo de encontrar os melhores pesos para a rede neural, de modo que ela possa realizar bem em tarefas específicas, como classificação ou regressão. Alguns exemplos são o SGD, Momentum e Adagrad.

O processo de inicialização em Deep Learning é o processo de definir os valores iniciais dos pesos das camadas da rede neural antes do treinamento começar. Alguns exemplos são inicialização aleatória, inicialização com zero e com pequenos valores próximo a zero.

## 1. Carga e instalação dos pacotes

In [1]:
# Versão da linguagem Python
from platform import python_version
print('A versão da linguagem python neste Jupyter Notebook é: ', python_version())

A versão da linguagem python neste Jupyter Notebook é:  3.9.13


In [2]:
# Instalação versão do torch
#!pip install -q torch==1.13.0

In [3]:
# Instalação versão do Pytorch
#!pip install -q torchvision==0.14.0

In [4]:
# Carga dos pacotes/funções

# Pacote para trabalhar no SO
import os

# Pacotes de visualização e Processamento de dados
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import shutil

# Pacote Torch
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler

# Pacote Torchvison
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
from torchvision import models

# Relatório do ambiente de desenvolvimento
import gc
import types
import pkg_resources
import pytorch_lightning as pl
%matplotlib inline

In [5]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Estudo de Caso 2 - Processos de Otimização/Inicialização" --iversions

Author: Estudo de Caso 2 - Processos de Otimização/Inicialização

matplotlib       : 3.5.2
pytorch_lightning: 1.8.3
torchvision      : 0.14.0
torch            : 1.13.0
numpy            : 1.21.5



## 2. Preparação dos Dados

### 2.1 Criação das pastas no diretório

In [6]:
pasta_dados_originais = 'data'

In [7]:
nomes_classes = ['cloudy', 'desert', 'green_area', 'water']

In [8]:
# Loop para criar as pastas
for i in range(len(nomes_classes)):
    
    # Extrai o nome de cada classe
    class1 = '/' + nomes_classes[i]
    
    # Cria as pastas
    os.makedirs('dados/treino/' + nomes_classes[i])
    os.makedirs('dados/val/' + nomes_classes[i])
    os.makedirs('dados/teste/' + nomes_classes[i])

### 2.2 Input das imagens nas pastas

In [9]:
# Loop
for k in range(len(nomes_classes)):
    
    # Extrai um nome de classe
    nome_classe = nomes_classes[k]
    
    # Define a fonte
    src = pasta_dados_originais + "/" + nome_classe 
    
    # Mostra qual classe estamos processando
    print("\nClasse:", nomes_classes[k])
    
    # Lista o conteúdo da pasta
    allFileNames = os.listdir(src)
    
    # "Embaralha" os dados
    np.random.shuffle(allFileNames)
    
    # Divisão = 70% treino, 15% teste, 15% validação 
    train_FileNames, val_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                          [int(len(allFileNames)*0.7), int(len(allFileNames)*0.85)])


    # Nome dos arquivos
    train_FileNames = [src + '/' + name for name in train_FileNames.tolist()]
    val_FileNames = [src + '/' + name for name in val_FileNames.tolist()]
    test_FileNames = [src + '/' + name for name in test_FileNames.tolist()]

    # Print
    print('Número Total de Imagens: ', len(allFileNames))
    print('Imagens de Treino: ', len(train_FileNames))
    print('Imagens de Validação: ', len(val_FileNames))
    print('Imagens de Teste: ', len(test_FileNames))
    
    # Copia as imagens
    for name in train_FileNames:
        shutil.copy(name, "dados/treino/" + nome_classe)
    
    for name in val_FileNames:
        shutil.copy(name, "dados/val/" + nome_classe)
        
    for name in test_FileNames:
        shutil.copy(name, "dados/teste/" + nome_classe)  


Classe: cloudy
Número Total de Imagens:  1500
Imagens de Treino:  1050
Imagens de Validação:  225
Imagens de Teste:  225

Classe: desert
Número Total de Imagens:  1131
Imagens de Treino:  791
Imagens de Validação:  170
Imagens de Teste:  170

Classe: green_area
Número Total de Imagens:  1500
Imagens de Treino:  1050
Imagens de Validação:  225
Imagens de Teste:  225

Classe: water
Número Total de Imagens:  1500
Imagens de Treino:  1050
Imagens de Validação:  225
Imagens de Teste:  225


### 2.3 Verificando o ambiente de desenvolvimento

In [10]:
# Relatório completo

# Verificando o dispositivo
processing_device = "cuda" if torch.cuda.is_available() else "cpu"

# Verificando se GPU pode ser usada (isso depende da plataforma CUDA estar instalada)
torch_aval = torch.cuda.is_available()

# Labels para o relatório de verificação
lable_1 = 'Visão Geral do Ambiente'
lable_2 = 'Se NVIDIA-SMI não for encontrado, então CUDA não está disponível'
lable_3 = 'Fim da Checagem'

# Função para verificar o que está importado nesta sessão
def get_imports():

    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):            
            name = val.__module__.split(".")[0]

        poorly_named_packages = {"PIL": "Pillow", "sklearn": "scikit-learn"}

        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name

# Imports nesta sessão
imports = list(set(get_imports()))

# Loop para verificar os requerimentos
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))
        
# Pasta com os dados (quando necessário)
pasta_dados = r'dados'

print(f'{lable_1:-^100}')
print()
print(f"Device:", processing_device)
print(f"Pasta de Dados: ", pasta_dados)
print(f"Versões dos Pacotes Requeridos: ", requirements)
print(f"Dispositivo Que Será Usado Para Treinar o Modelo: ", processing_device)
print(f"CUDA Está Disponível? ", torch_aval)
print("Versão do PyTorch: ", torch.__version__)
print("Versão do Lightning: ", pl.__version__)
print()
print(f'{lable_2:-^100}\n')
!nvidia-smi
gc.collect()
print()
print(f"Limpando a Memória da GPU (se disponível): ", torch.cuda.empty_cache())
print(f'\n{lable_3:-^100}')

--------------------------------------Visão Geral do Ambiente---------------------------------------

Device: cpu
Pasta de Dados:  dados
Versões dos Pacotes Requeridos:  [('matplotlib', '3.5.2'), ('numpy', '1.21.5'), ('torch', '1.13.0'), ('torchvision', '0.14.0')]
Dispositivo Que Será Usado Para Treinar o Modelo:  cpu
CUDA Está Disponível?  False
Versão do PyTorch:  1.13.0+cpu
Versão do Lightning:  1.8.3

------------------Se NVIDIA-SMI não for encontrado, então CUDA não está disponível------------------


Limpando a Memória da GPU (se disponível):  None

------------------------------------------Fim da Checagem-------------------------------------------


'nvidia-smi' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.


### 2.4 Funções auxiliares para input dos dados no dispositivo de treino

In [11]:
# Função para obter o device
# Se disponível usamos GPU, caso contrário usamos CPU.

def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

In [12]:
# Função para enviar um tensor para o device
def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking = True)

In [13]:
# Classe para enviar os dataloaders para o device
class DeviceDataLoader():
    
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
    
    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)
    
    def __len__(self):
        return len(self.dl)

In [14]:
# Visualiza o device
device = get_default_device()
device

device(type='cpu')

## 3. Pré-Processamento dos Dados

### 3.1 Transformação dos dados

In [15]:
# Processo de normalização dos dados (médias e desvio padrão para cada canal de cor)
stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))

In [16]:
# Define as transformações dos dados
transforms = transforms.Compose([transforms.RandomCrop(64, padding_mode = 'reflect'),
                                 transforms.Resize(64),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 transforms.Normalize(*stats, inplace = True)])

### 3.2 Carga dos dados

In [17]:
# Dados de treino
dados_treino = datasets.ImageFolder('dados/treino', transform = transforms)

In [18]:
# Dados de validação
dados_valid = datasets.ImageFolder('dados/val', transform = transforms)

In [19]:
# Dados de teste
dados_teste = datasets.ImageFolder('dados/teste', transform = transforms)

In [20]:
# Visualiza os dados
print(f"Número Total de Imagens de Treino: {len(dados_treino)}")
print(f"Número Total de Imagens de Validação: {len(dados_valid)}")
print(f"Número Total de Imagens de Teste: {len(dados_teste)}") 

Número Total de Imagens de Treino: 3941
Número Total de Imagens de Validação: 845
Número Total de Imagens de Teste: 845


### 3.3 Prepara os dataloaders

In [21]:
# Parâmetros para os dataloaders
batch_size = 8
shuffle = True
num_workers = 2
pin_memory = True

In [22]:
# Data loader de treino
data_loader_treino = DataLoader(dados_treino, 
                                batch_size, 
                                shuffle = shuffle, 
                                num_workers = num_workers, 
                                pin_memory = pin_memory)

In [23]:
# Data loader de validação
data_loader_valid = DataLoader(dados_valid, 
                               batch_size, 
                               num_workers = num_workers, 
                               pin_memory = pin_memory)

In [24]:
# Data loader de teste
data_loader_teste = DataLoader(dados_teste, 
                               batch_size * 2, 
                               num_workers = num_workers, 
                               pin_memory = pin_memory)

In [25]:
# Função para "desnormalizar" os dados
def denormalize(images, means, stds):
    means = torch.tensor(means).reshape(1, 3, 1, 1)
    stds = torch.tensor(stds).reshape(1, 3, 1, 1)
    return images * stds + means

In [26]:
# Função para imprimir um batch de dados
def mostra_dados(dl):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize = (12, 12))
        ax.set_xticks([]); ax.set_yticks([])
        denorm_images = denormalize(images, *stats)
        ax.imshow(make_grid(denorm_images[:64], nrow = 8).permute(1, 2, 0).clamp(0,1))
        break

In [27]:
#mostra_dados(data_loader_treino)

## 4. Arquitetura do Modelo